# Studio Preliminare
#### Filippo Iacobelli e Luca Rossicone

## Lista delle primitive interne al modulo Mapper.jl

### Primitive sulle curve

```@docs
Lar.circle
```

```@docs
Lar.helix
```



### Primitive di superficie

```@docs
Lar.disk
```

```@docs
Lar.helicoid
```

```@docs
Lar.ring
```

```@docs
Lar.cylinder
```

```@docs
Lar.sphere
```

```@docs
Lar.toroidal
```



### Primitive solide

```@docs
Lar.cuboid
```

```@docs
Lar.ball
```

```@docs
Lar.hollowCyl
```

```@docs
Lar.hollowBall
```

```@docs
Lar.torus
```



### Funzioni ausiliarie

```@docs
Lar.approxVal
```

```@docs
Lar.simplyfyCells
```


## Lista delle primitive esterne al modulo Mapper.jl

```@docs
Lar.cuboidGrid
```

```@docs
Lar.simplexGrid
```
```@docs
Lar.apply
```

```@docs
Lar.t
```

```@docs
Lar.s
```

## Protipi di funzioni da ottimizzare

In [1]:
using DataStructures
using BenchmarkTools
using LinearAlgebraicRepresentation
using ViewerGL
using IntelVectorMath
using LinearAlgebra
Lar = LinearAlgebraicRepresentation
GL = ViewerGL
LA = LinearAlgebra
IVM = IntelVectorMath


IntelVectorMath

In [2]:
function simplifyCells(V,CV)
	PRECISION = 5
	vertDict = DefaultDict{Array{Float64,1}, Int64}(0)
	index = 0
	W = Array{Float64,1}[]
	FW = Array{Int64,1}[]

	for incell in CV
		outcell = Int64[]
		for v in incell
			vert = V[:,v]
			key = map(Lar.approxVal(PRECISION), vert)
			if vertDict[key]==0
				index += 1
				vertDict[key] = index
				push!(outcell, index)
				push!(W,key)
			else
				push!(outcell, vertDict[key])
			end
		end
		append!(FW, [[Set(outcell)...]])
	end
	return hcat(W...),FW
end

simplifyCells (generic function with 1 method)

In [3]:
function simplifyCellsOpt(V,CV)::Tuple{Matrix{Float64},Vector{Vector{Int64}}}
	PRECISION = 5
	vertDict = DefaultDict{Array{Float64,1}, Int64}(0)
	index = 0
	W = Array{Float64,1}[]
	FW = Array{Int64,1}[]

	@inbounds @simd for incell in CV
		outcell = Int64[]
		@inbounds @simd for v in incell
			vert = @view V[:,v]
			key = map(Lar.approxVal(PRECISION), vert)
			if vertDict[key]==0
				index += 1
				vertDict[key] = index
				push!(outcell, index)
				push!(W,key)
			else
				push!(outcell, vertDict[key])
			end
		end
		append!(FW, [[Set(outcell)...]])
	end
	return hcat(W...),FW
end

simplifyCellsOpt (generic function with 1 method)

In [4]:
V, EV = Lar.cuboidGrid([36])
@btime simplifyCells(V, EV)
V, EV = Lar.cuboidGrid([36])
@btime simplifyCellsOpt(V, EV)

  76.172 μs (853 allocations: 57.95 KiB)


  73.244 μs

 (998 allocations: 63.61 KiB)


([0.0 1.0 … 35.0 36.0], [[2, 1], [2, 3], [4, 3], [5, 4], [5, 6], [6, 7], [7, 8], [9, 8], [10, 9], [11, 10]  …  [27, 28], [29, 28], [29, 30], [31, 30], [32, 31], [32, 33], [34, 33], [34, 35], [35, 36], [36, 37]])

In [5]:
function circle(radius=1., angle=2*pi)
    function circle0(shape=[36])
        V, EV = Lar.cuboidGrid(shape)
        V = (angle/shape[1])*V
        V = hcat(map(u->[radius*cos(u); radius*sin(u)], V)...)
        W, EW = simplifyCells(V, EV)
        return W, EW
    end
    return circle0
end

circle (generic function with 3 methods)

In [6]:
function circleOpt(radius=1., angle=2*pi)
    function circle0(shape=[36])
        V, EV = Lar.cuboidGrid(shape)
        V = (angle/shape[1])*V
        V = vcat(radius*IVM.cos(V),radius*IVM.sin!(V))
        W, EW = simplifyCellsOpt(V, EV)
        return W, EW
    end
    return circle0
end

circleOpt (generic function with 3 methods)

In [7]:
@btime circle()()
@btime circleOpt()()

  169.015 μs (2134 allocations: 149.92 KiB)


  163.705 μs (2239 allocations: 151.88 KiB)


([1.0 0.98481 … 0.93969 0.98481; 0.0 0.17365 … -0.34202 -0.17365], [[2, 1], [2, 3], [4, 3], [5, 4], [5, 6], [6, 7], [7, 8], [9, 8], [10, 9], [11, 10]  …  [27, 28], [29, 28], [29, 30], [31, 30], [32, 31], [32, 33], [34, 33], [34, 35], [35, 36], [36, 1]])

In [8]:
# GL.VIEW([
# 	GL.GLLines(circleOpt()()..., GL.COLORS[6]),
# 	GL.GLFrame
# ]);

In [9]:
function toroidal(r=1., R=2., angle1=2*pi, angle2=2*pi)
  function toroidal0(shape=[24, 36])
      V, CV = Lar.simplexGrid(shape)
      V = [angle1/(shape[1]) 0;0 angle2/(shape[2])]*V
      W = [V[:, k] for k=1:size(V, 2)]
      V = hcat(map(p->let(u, v)=p;[(R+r*cos(u))*cos(v);
        (R+r*cos(u))*sin(v);-r*sin(u)]end, W)...)
      W, CW = simplifyCells(V, CV)
      return W, CW
  end
  return toroidal0
end

toroidal (generic function with 5 methods)

In [10]:
function toroidalOpt(r=1., R=2., angle1=2*pi, angle2=2*pi)
    function toroidal0(shape=[24, 36])
        V, CV = Lar.simplexGrid(shape)
        V = [angle1/(shape[1]) 0;0 angle2/(shape[2])]*V
        U = V[1,:]; Z = V[2,:]
        sinU = IVM.sin(U); sinZ = IVM.sin(Z)
        IVM.cos!(U); IVM.cos!(Z)
        tmp = r*U.+R
        V = hcat(tmp.*Z, tmp.*sinZ, -r*sinU)
        W, CW = simplifyCellsOpt(V', CV)
        return W, CW
    end
    return toroidal0
end

toroidalOpt (generic function with 5 methods)

In [11]:
@btime toroidal()()
@btime toroidalOpt()()

  7.312 ms (68794 allocations: 4.80 MiB)


  6.914 ms (75567 allocations: 4.93 MiB)


([3.0 2.96593 … 2.82248 2.92087; 0.0 0.0 … -0.49768 -0.51503; 0.0 -0.25882 … 0.5 0.25882], [[2, 3, 1], [4, 2, 3], [5, 4, 2], [5, 4, 6], [5, 6, 7], [6, 7, 8], [7, 9, 8], [10, 9, 8], [11, 10, 9], [11, 10, 12]  …  [39, 861, 860], [39, 41, 861], [41, 861, 862], [41, 43, 862], [43, 863, 862], [43, 45, 863], [45, 863, 864], [47, 45, 864], [841, 47, 864], [841, 47, 1]])

In [12]:
# GL.VIEW([
# 	GL.GLGrid(toroidalOpt()()..., GL.COLORS[1]),
# 	GL.GLFrame
# ]);

In [13]:
function cuboid(maxpoint::Array, full=false,
    minpoint::Array=zeros(length(maxpoint)))
    @assert( length(minpoint) == length(maxpoint) )
    dim = length(minpoint)
    shape = ones(Int, dim)
    cell = Lar.cuboidGrid(shape, full)
    size = maxpoint - minpoint
    out = Lar.apply(Lar.t(minpoint...) * Lar.s(size...), cell)
end


cuboid (generic function with 3 methods)

In [14]:
function cuboidOpt(maxpoint::Array, full=false,
    minpoint::Array=zeros(length(maxpoint)))::
    Tuple{Matrix{Float64},Vector{Vector{Int64}}}
    @assert length(minpoint) == length(maxpoint)
    dim = length(maxpoint)
    shape = ones(Int, dim)
    cell = Lar.cuboidGrid(shape, full)
    out = Lar.apply(Lar.t(minpoint...) * Lar.s(maxpoint...), cell)
end

cuboidOpt (generic function with 3 methods)

In [25]:
@btime cuboid([1,1,1])
@btime cuboidOpt([1,1,1])

  19.505 μs (286 allocations: 21.64 KiB)


  18.484 μs (282 allocations: 21.48 KiB)


([0.0 0.0 … 1.0 1.0; 0.0 0.0 … 1.0 1.0; 0.0 1.0 … 0.0 1.0], [[1, 2, 3, 4, 5, 6, 7, 8]])

In [16]:
# V, (VV, EV, FV, CV) = Lar.cuboid([1,1,1], true);
# assembly = Lar.Struct([ (V, CV), Lar.t(1.5,0,0), (V, CV) ])
# GL.VIEW([
# 	GL.GLPol( Lar.struct2lar(assembly)..., GL.COLORS[1],0.75 ),
# 	GL.GLFrame 
# ]);

## Conclusioni

È facile osservare come i vantaggi ottenuti con gli accorgimenti effettuati portono a benefici irrisori in termini tempistici.
Questo comportamento si può facilmente giustificare effettuando il profiling delle funzioni e riscontrando che i calcoli maggiormente onerosi sono dovuti a funzioni esterne (in particolare `cuboidGrid` e `simplexGrid`).

In [24]:
using Profile
toroidal()()
@profile toroidal()()
Profile.print(format=:flat)

 Count  Overhead File                    Line Function
 =====  ======== ====                    ==== ========
    44         0 @Base/Base.jl            384 include(mod::Module, _path::Str...
    16        16 @Base/abstractarray.jl  1167 getindex(::Matrix{Float64}, ::F...
     1         0 @Base/abstractarray.jl  1170 getindex
     1         0 @Base/abstractarray.jl  1170 getindex(::Matrix{Float64}, ::F...
     3         0 @Base/abstractarray.jl  2395 hash(A::Vector{Float64}, h::UIn...
     1         0 @Base/abstractarray.jl  2399 hash(A::Vector{Float64}, h::UIn...
    11         3 @Base/abstractarray.jl  2294 map(f::Function, A::Vector{Floa...
     2         0 @Base/abstractarray.jl   740 similar
     4         0 @Base/abstractdict.jl    507 get!
     1         1 @Base/array.jl             ? _collect(c::Vector{Float64}, it...
     3         0 @Base/array.jl           695 _collect(c::Vector{Float64}, it...
     4         0 @Base/array.jl           700 _collect(c::Vector{Float64}, it...
 

[*Link al repository del progetto*](https://github.com/Asprofumo/mapper.jl/)
